In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets 
import torchvision.transforms as transform
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm


/home/maksm/miniconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
class CNN(nn.Module):
    def __init__(self, input_chanels=1, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(input_chanels, 30, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(30, 15, kernel_size=3, stride=1, padding=1)
        self.fc = nn.Linear(7 * 7 * 15, num_classes)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        out = self.fc(x)
        return out

In [4]:
load_train = datasets.MNIST('datasets/', transform=transform.ToTensor())
load_test = datasets.MNIST('datasets/', transform=transform.ToTensor(), train=False)


In [14]:
batch_sizes = [256]
learning_rates = [1e-3]
classes = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]

In [21]:
num_epoch = 1

for batch_size in batch_sizes:
    for learning_rate in learning_rates:
        model = CNN().to(device)

        optimizer = optim.Adam(model.parameters(), learning_rate)
        criterion = nn.CrossEntropyLoss()

        train = DataLoader(load_train, batch_size, shuffle=True)
        test = DataLoader(load_test, batch_size, shuffle=True)

        writer = SummaryWriter(f"runs/MNIST/MiniBatchSize {batch_size} LR {learning_rate}")

        images, label = next(iter(train))
        writer.add_graph(model, images.to(device))

        for epoch in range(num_epoch):
            for batch_idx, (x_batch, y_batch) in enumerate(tqdm(train)):
                
                # Обучение
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)

                y_pred_batch = model(x_batch)

                loss = criterion(y_pred_batch, y_batch)
                optimizer.zero_grad()

                loss.backward()
                optimizer.step()
                # Конец обучения

                # Формируем TensorBoard
                img_grid = torchvision.utils.make_grid(x_batch)
                _, predictions = y_pred_batch.max(1)
                features = x_batch.reshape(x_batch.shape[0], -1)
                class_labels = [classes[label] for label in predictions]

            
                if batch_idx % 20 == 0:
                    writer.add_scalar('Loss', loss, global_step=batch_idx)
                
                if batch_idx == 230:
                    writer.add_embedding(
                        features,
                        metadata=class_labels,
                        label_img=x_batch,
                        global_step=batch_idx,
                    )

                writer.add_image('mnist_images', img_grid, global_step=batch_idx)
                
            

100%|██████████| 235/235 [00:30<00:00,  7.78it/s]


In [ ]:
def check_accuracy(model, data):
    model.eval()

    num_samples = 0
    num_correct = 0

    with torch.no_grad():

        for x_batch, y_batch in data:

            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)

            y_pred = model(x_batch)
            _, y_pred = y_pred.max(1)

            num_correct += (y_pred == y_batch).sum()
            num_samples += y_pred.shape[0]

    accuracy = num_correct / num_samples * 100
    model.train()
    print(accuracy, )

In [ ]:
check_accuracy(model, test)